In [ ]:
from IPython.display import Markdown

from datetime import datetime, timedelta
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import curve_fit
import suncalc

tz='Europe/Berlin'

# Create a connection to your PV database. 
# Make sure you installed the schema as described in the README of the pv project.
db = duckdb.connect('pv.db')

# A snapshot of my databases content can also be retrieved with DuckDB running in-memory in this very notebook itself.
# It works by installing the httpfs extension and than running a database import
# db = duckdb.connect(database=':memory:')
# db.sql('INSTALL httpfs')
# db.sql('LOAD httpfs')
# db.sql("IMPORT database 'http://simons.ac/pv'")

# Photovoltaics at simons.ac

In [ ]:
Markdown(f"""
Last update: _{datetime.now().strftime('%c')}_.

The sources, including the SQL queries and solar course computations are available as a Jupyter notebook in my pv repository: [michael-simons/pv](https://github.com/michael-simons/pv).
""")

## Technical background

This is a quick overview over our photovoltaics installation in Aachen. We started planning this in September 2022. Originally, I didn't want to put too much thought and effort into analyzing the result, but it turned out: The smart devices vendors sell you are still not that smart, even in 2023. So here I am, collecting measurements straight from our SunSpec compatible inverter over Modbus and dumping it straight into a [DuckDB](https://duckdb.org) analytical database. The [logger](https://github.com/michael-simons/pv/tree/main/logger) I created for that purpose is based on the [Energy systems reading toolkit](https://energy.basjes.nl) by [Niels Basjes](https://mastodon.basjes.nl/@niels). 

The database design follows a subset of the [pink database design](https://www.salvis.com/blog/2018/07/18/the-pink-database-paradigm-pinkdb/#feature_3). Instead of writing down all the analytic queries straight into the notebook or my programming environment of choice, I created a bunch of views that basically represents the api of my scheme. Please visit the above lined repository for more information.

## General reasons

As both gas and electricy prices have been erratic the last years, even more so since the Russian invasion into Ukraine since February 2022 and these days with the increasing inflation, we wanted to invest into a somewhat more stable price in the long run. As reference, I added the current prices we have for buying a kWh electric energy into the database. The price has seen a whooping increase of more than 75% since 2010:

In [ ]:
buying_prices = db.sql("SELECT valid_from AS year, net FROM buying_prices").df()
imported = db.sql("SELECT date_trunc('year', period_end) as year, import FROM official_measurements ORDER by period_start ASC").df()

imported_with_price = pd.merge_asof(buying_prices, imported, on="year")

_, ax = plt.subplots()
imported_with_price.plot(x = 'year', y = 'net',    ylabel='Net price in ct/kWh', ax = ax)
imported_with_price.plot(x = 'year', y = 'import', ylabel='Imported energy in kWh', ax = ax, secondary_y = True)

plt.show()

The amount of energy we use has been relatively constant over the years. There's a slight increase with me working remotely since 2018 and the kids playing more and more video games these days, but that's neglectable. 

## The setup

We went for an installation of 10.53kWp, in a split east/west setup. In total we have 26 [Solarwatt Glass-Glass "Panel Vision AM 4.0"](https://solarwatt.canto.global/direct/document/2bp9ip7a492p51429ek8qdr706/HPyRw7XsY0A3hu2B1c6SpNGxiOc/original?content-type=application%2Fpdf&name=Datenblatt+SOLARWATT+Panel+vision+AM+4.0+pure+de.pdf) modules with an output of 405Wp each, in plus selection:

* 14 directed east
* 12 directed west

The inverter is a Kaco [blueplanet 10.0 NX3 M2](https://kaco-newenergy.com/de/produkte/blueplanet-3.0-20.0-NX3-M2/). The possible peak output ov 10.53kWp is a lot more than we would actually use in a year, but we wanted to be prepared for the future. We haven't yet decided to replace our 10+ year old car with an electric one or if we go straight for a stationary battery solution followed by replacing the heating system with a heatpump eventually. Time will tell. Until then, let's have a look at the current results:

## The results

### Energy production

In [ ]:
period = db.sql('SELECT min(measured_on) AS min, max(measured_on) AS max FROM measurements').df()
Markdown(f"""
The measurement in the currently loaded dataset spans right now from **{period['min'][0].strftime('%Y-%m-%d')}** to **{period['max'][0].strftime('%Y-%m-%d')}** and in that period we had the following overall production:
""")

In [ ]:
db.sql('SELECT * FROM overall_production').df()

with averages per month as follows:

In [ ]:
average_per_month = db.sql('SELECT * FROM average_production_per_month').df()[['Month', 'kWh']].dropna()
average_per_month.plot(kind='bar', x='Month')
plt.show()

As of writing, we still haven't had any day really close to possible peak. However, our original assumption that we are better of using both side of the house holds true. The inverter boots up pretty close to sunrise and already in May we had several evenings where the whole setup was producing enough energy for late cooking. Plotting the production per hour shows that:

In [ ]:
average_per_hour = db.sql('SELECT * FROM average_production_per_hour').df()[['Hour', 'kWh']].dropna()
average_per_hour.plot(kind='barh', x='Hour').invert_yaxis()
plt.show()

The production for the best performing day so far looks like this:

In [ ]:
best_day = db.sql('SELECT * FROM best_performing_day').df()
lat_long = db.sql('SELECT * FROM place_of_installation').df()
lat, long = lat_long['lat'][0], lat_long['long'][0]

localize_tz = lambda v: v.tz_localize(tz=tz)
to_degree = lambda v: math.degrees(v) if v > 0 else 0

best_day['measured_on'] = best_day['measured_on'].apply(localize_tz)
best_day['altitude'] = pd.DataFrame(suncalc.get_position(best_day['measured_on'], long, lat)['altitude'].apply(to_degree))

_, ax = plt.subplots(figsize=plt.figaspect(1/2))
best_day.plot(x = 'measured_on', y='production', ylabel='W', ax = ax)
best_day.plot(x = 'measured_on', y='altitude', ylabel='Sun altitude in °', ax = ax, secondary_y = True,  mark_right=False)

sun_times = suncalc.get_times(best_day['measured_on'][0].floor('d') + timedelta(days=1), long, lat)
sun_times = {k: v.tz_localize(tz='UTC').tz_convert(tz='Europe/Berlin') for k, v in sun_times.items()}

ax.axvspan(sun_times['dawn'], sun_times['sunrise_end'], color='red', alpha=0.1)
ax.axvspan(sun_times['solar_noon'] - timedelta(minutes=5), sun_times['solar_noon'] + timedelta(minutes=5), color='yellow', alpha=0.4)
ax.axvspan(sun_times['sunset_start'], sun_times['dusk'], color='blue', alpha=0.1)

plt.show()

The orange chart is the sun position at our location, the red bar the time from dawn until sunrise end, the yellow bar a 10 minute period around true or solar noon, the blue bar finally is the time period from sunset start until dusk.

I personally liked the visualization that [Oli](https://social.tchncs.de/@oli) did at [Ein Jahr Photovoltaik: "Tolle Dinge, die man mit SQL machen kann"](https://tonick.net/p/2022/12/ein-jahr-photovoltaik/) so much, that I wanted to have this too. Turns out, it is easy enough with DuckDB 0.8.0 and its excellent support for the `PIVOT` statement (See the corresponding view [`average_production_per_month_and_hour`](https://github.com/michael-simons/pv/blob/main/sql/schema/R__Create_view_average_production_per_month_and_hour.sql). Together with the "Styler" feature of Pandas and the integraded color maps of matplotlib you get a colorized table based on the value of the cells with zero effort. In the below example I drop the original dataframe index column and use my `Hour` column instead. It is on purpose that it doesn't get a color.

In [ ]:
df = db.sql('SELECT * FROM average_production_per_month_and_hour').df().set_index(["Hour"]).fillna(0)
df.style.background_gradient(cmap="YlOrRd")

A sun course diagram drawn for the location of our house shows that we won't have much output in the winter times, basically some sune from between 9:00 in the morning until 17:00 in the afternoon, at an angle that is probably not very productive. The below diagram has 4 plots: The course of the sun at winter and summer solstice plus the most productive day in the same colors as above and March 21st in grey as a reference value in between the two extremes. The fancy things shaped like eights are called [Analemma](https://en.wikipedia.org/wiki/Analemma); a figure that is the result of observing the variying course of the sun from a fixed location on earth at the same solar time over the year:

In [ ]:
dt = datetime.today()
dt = datetime(dt.year, 1, 1)

times = pd.date_range(datetime(dt.year, 1, 1), datetime(dt.year+1, 1, 1), inclusive='left', freq='H', tz=tz)
solpos = pd.DataFrame(suncalc.get_position(times, long, lat))
solpos = pd.DataFrame(times).join(solpos, how='inner').loc[solpos['altitude'] > 0, :]
solpos = solpos.set_index(0)
solpos['altitude'] = np.vectorize(to_degree)(solpos['altitude'])
solpos['azimuth'] = np.vectorize(lambda v: math.degrees(v))(solpos['azimuth'])

fig, ax = plt.subplots(figsize=plt.figaspect(1/3))
points = ax.scatter(solpos['azimuth'], solpos['altitude'], s=2, c=solpos.index.dayofyear, label=None)
fig.colorbar(points)

for hour in np.unique(solpos.index.hour):
    subset = solpos.loc[solpos.index.hour == hour, :]
    height = subset['altitude']
    pos = solpos.loc[height.idxmax(), :]
    ax.text(pos['azimuth'], pos['altitude'], str(hour))
    
cmap = plt.colormaps.get_cmap('YlOrRd')
neutral_day = datetime(dt.year, 3, 21)
dates_of_interest = [datetime(dt.year-1, 12, 21), best_day['measured_on'][0].tz_localize(None).to_pydatetime(), neutral_day, datetime(dt.year, 6, 21)]
dates_of_interest.sort()
dates_of_interest[0] = dates_of_interest[0].replace(year=dt.year)

for index, date in enumerate(pd.to_datetime(dates_of_interest)):
    times = pd.date_range(date, date+pd.Timedelta('24h'), freq='1min', tz=tz)
    solpos = pd.DataFrame(suncalc.get_position(times, long, lat))
    solpos = solpos.loc[solpos['altitude'] > 0, :]
    solpos['altitude'] = np.vectorize(to_degree)(solpos['altitude'])
    solpos['azimuth'] = np.vectorize(lambda v: math.degrees(v))(solpos['azimuth'])
    label = date.strftime('%Y-%m-%d')
    color = 'grey' if date == neutral_day else cmap(0.4 + 0.2 * index)
    ax.plot(solpos['azimuth'], solpos['altitude'], label=label, color=color)
    
fig.legend(loc='center right')
ax.set_xlabel('Solar Azimuth (degrees, measured from south to west)')
ax.set_ylabel('Solar Elevation (degrees)')

plt.show()

### Money and amortization

Sadly, we don't have full import / export data right from the start (did I mention smart devices suck?), so for the first month I have no real compensation for feeding back into the grid. Therefor we only have the hypothetical full export back into the grid for the first couple of months. The actual partial export computes the amortization by the compensation we get plus the money we saved for not having to import energy. The compensation for a full export to the grid is slightly higher than for a partial one. However, as of writing, I buy nearly for tripple the amount than I could sell at, even for the full export compensation. So, taking the savings into account leads to a faster and more sane amortization.

#### Actually amortization

The chart represents the running sum of the initial invest plus the money made respectively saved per month. The area in light purple is linearly extrapolated. Kiss your expectations of getting rich fast with PV goodbye while looking at this chart, unless you have some acres of unused ground somewhere and can think a bit bigger.

In [ ]:
# Idea of the linear extrapolation is mostly from this great answer by tmthydvnprt
# https://stackoverflow.com/a/35959909/1547989

df = db.sql("SELECT month, full_export FROM amortization").df()
df.set_index(["month"], inplace=True)

_, ax = plt.subplots(figsize=plt.figaspect(1/3))

legend = ["Full export", "Break even"]
# Memorize last entry
last_month = df.index[len(df)-1]
first_extrapolated_month = None

if len(df) < 120:    
    num_months_extrapolation = 120 - len(df)
    
    # Extend the index
    df = pd.DataFrame(data=df, index=pd.date_range(
            start=df.index[0],
            periods=len(df.index) + num_months_extrapolation,
            freq='MS'
        )
    )
    
    first_extrapolated_month = last_month + pd.DateOffset(months=1);
    # Duplicate the first extrapolated month to have enough supporting points
    df.loc[first_extrapolated_month] = df.loc[last_month]
    
    di = df.index
    df = df.reset_index(drop=True)
    
    # Function to curve fit to the data
    def func(x, a, b):
        return a * x + b
    
    # Initial parameter guess, just to kick off the optimization
    guess = (1, 1)
    
    # Create copy of data to remove NaNs for curve fitting
    fit_df = df.dropna()
    
    # Place to store function parameters for each column
    col_params = {}
    
    # Curve fit each column
    for col in fit_df.columns:
        # Get x & y
        x = fit_df.index.astype(float).values
        y = fit_df[col].values
        # Curve fit column and get curve parameters
        params = curve_fit(func, x, y, guess)
        # Store optimized parameters
        col_params[col] = params[0]
    
    # Extrapolate each column
    for col in df.columns:
        # Get the index values for NaNs in the column
        x = df[pd.isnull(df[col])].index.astype(float).values
        # Extrapolate those points with the fitted function
        df[col][x] = func(x, *col_params[col])
    
    df.index = di
    legend = legend + ["Actual data", "Extrapolated data"]


df.plot(figsize=plt.figaspect(1/2), ax=ax)
ax.axhline(color='green')
if first_extrapolated_month != None:
    ax.axvspan(df.index[0], first_extrapolated_month, color='green', alpha=0.1)
    ax.axvspan(first_extrapolated_month, df.index[-1], color='purple', alpha=0.05)
ax.legend(legend , loc='lower right')
    
plt.xlabel("Time in production")
plt.ylabel("Accumulated invest / return in €")
plt.ylim(top=1000)

plt.show()